In [184]:
import pandas as pd

from PIL import Image
#!pip install opencv-python
from PIL import Image
import collections
import math

In [237]:
def crop_image(og_image, DALLE_image):
    """
    Crops the DALLE image according to the dimensions of the original image 
    Args: 
        og_image: filename of original image (assumed to be located in...)
        DALLE_image: filename to DALLE_image (assumed to be located in...)

    Returns: 
        path of new cropped DALLE image file 

    """

    # need to edit this based on file path stuff... 
    og_image_file = og_image
    og_image = Image.open(og_image_file)

    # Get dimensions of the original image
    width_og, height_og = og_image.size

    # need to edit this based on file path stuff... 
    og_image_file = DALLE_image
    DALLE_image = Image.open(og_image_file)

    # get dimensions of DALLE image
    width, height = DALLE_image.size

    # Calculate the dimensions for cropping
    width_diff = width - width_og 
    height_diff = height - height_og 
    width_diff_one_side = width_diff / 2
    height_diff_one_side = height_diff / 2
    print(width_diff_one_side)
    print(height_diff_one_side)

    # Ensure the image is larger than the original image
    if width_diff < 0 or height_diff < 0:
        print("Error: Cropped area is larger than the original image.")
    else:
        # calculate the cropping coordinates
        left = width_diff_one_side
        right = width - width_diff_one_side
        top = height_diff_one_side
        bottom = height -  height_diff_one_side

        print(left, right, top, bottom)

        # ensure that the images are the same size even if dimensions 
        # are an odd num 
        if (width_diff_one_side % 2 != 0):
            math.floor(left)
            math.floor(right)
        if (height_diff_one_side % 2 != 0):
            math.floor(bottom)
            math.floor(top)

        # perform the cropping
        cropped_img = img.crop((left, top, right, bottom))
        width, height = cropped_img.size
    
        # Display the cropped image
        print(width, height)
        cropped_img.save("cropped_image.jpg")


In [238]:
def get_image_pixels(image_file):
    # Open the image
    im = Image.open(image_file)

    # Load pixel data
    pix = im.load()

    # Get the width and height of the image
    width, height = im.size

    # Create a 2D array to store pixel values
    pixels_2d = [[0 for _ in range(width)] for _ in range(height)]

    # Iterate over each pixel and store its RGBA values in the 2D array
    for x in range(width):
        for y in range(height):
            pixels_2d[y][x] = pix[x, y]

    return pixels_2d


In [3]:
image_path = "DALLE_images/Adam_Koester_6_oval_DALLE.jpg"
file_name = image_path.split("/")[1]
path = file_name.split(".jpg")
transformed_image_path = "TRANSFORMED_DALLE_images/" + path[0] + "_TRANSFORMED.jpg"
transformed_image_path

'TRANSFORMED_DALLE_images/Adam_Koester_6_oval_DALLE_TRANSFORMED.jpg'

In [256]:
def create_image_from_arr(image_path, pixels_2d):
    # Assuming pixels_2d is your 2D array of pixels
    # Example pixels_2d = [[(255, 255, 255), (0, 0, 0), ...], [...], ...]

    # Assuming width and height are the dimensions of your image
    width, height = len(pixels_2d[0]), len(pixels_2d)

    # Create a new image
    new_im = Image.new("RGB", (width, height))

    # Load pixel data
    pix = new_im.load()

    # Set pixel values
    for x in range(width):
        for y in range(height):
            pix[x, y] = pixels_2d[y][x]

    # Save the image to a file
    path = image_path.split(".jpg")
    transformed_image_path = path[0] + "_TRANSFORMED.jpg"
    new_im.save(transformed_image_path)  # Change the filename and extension as needed
    return transformed_image_path

In [257]:
def get_oval(og_image_file, AI_image_file):
    """
    Uses breadth-first search to identify the white oval frame in the og image and apply it to the AI generated image
    og_image_file, AI_image_file
    """

    # getting the 2d array of pixels from each image
    og_image_pixels = get_image_pixels(og_image_file)
    AI_image_pixels = get_image_pixels(AI_image_file)

    rows, cols = len(og_image_pixels), len(og_image_pixels[0])
    rows1, cols1 = len(AI_image_pixels), len(AI_image_pixels[1])
    
    visited = set()

    def bfs(r, c):
        q = collections.deque()
        visited.add((r, c))
        q.append((r, c))

        while q:
            row, col = q.popleft()
            directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

            for dr, dc in directions:
                r, c = row + dr, col + dc
                if (r in range(rows) 
                    and c in range(cols) 
                    and og_image_pixels[r][c] == (255, 255, 255)
                    and (r, c) not in visited):
                    q.append((r, c))
                    visited.add((r, c))
                    AI_image_pixels[r][c] = (255, 255, 255)

    # breadth first search starting from the corners of the image (to capture the oval)
    bfs(0, 0)
    bfs(rows - 1, 0)
    bfs(0, cols - 1)
    bfs(rows - 1, cols - 1)
    output_image_path = create_image_from_arr(AI_image_file, AI_image_pixels)
    return output_image_path


    

In [258]:
get_oval("testing_image.jpg", "cropped_image.jpg")

'cropped_image_TRANSFORMED.jpg'